In [66]:
# import
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score
import os
import math
# label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================

def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type   

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            if label != "":
                dataset.append(
                                {   
                                    'uuid': root.split("\\")[-1],
                                    'label': label,
                                    'image_path': os.path.join(root, file)
                                }
                            )

    return dataset

In [67]:
import matplotlib.pyplot as plt
import cv2
def plot_img(image_path):
    print(image_path)
    # image = cv2.imread(image_path)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV 讀取的圖像是 BGR 需要轉為 RGB
    image1 = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    image2 = cv2.imdecode(np.fromfile(image_path.replace("front", "side"), dtype=np.uint8), cv2.IMREAD_COLOR)



    # **顯示圖片**
    plt.figure(figsize=(8, 5))  # 設定圖片大小

    # 顯示第一張圖（front）
    plt.subplot(1, 2, 1)  # (行數, 列數, 當前索引)
    plt.imshow(image1)
    plt.title("AP(Mortise) View")
    plt.axis("off")  # 隱藏座標軸

    # 顯示第二張圖（side）
    plt.subplot(1, 2, 2)
    plt.imshow(image2)
    plt.title("Lateral View")
    plt.axis("off")

    plt.suptitle(image_path.split("\\")[-1])

    plt.show()  # 顯示圖片





In [68]:
## 參數設置
image_dir = "E:\\data_bone\\11-a+b_swift_cut_正確_V2_踢盲檢\\front"
image_dir2 = "E:\\data_bone\\11-只有盲檢\\front"
concat_type = "concat1_踢盲檢_測盲測"
class_count = 3
save_cam_path = "D://reaserch//Bone-Fracture-Detection//concat//2-concat1//踢盲檢_測盲測_cam//"

In [69]:
## load data and  labels
# =========================
data = load_path(image_dir, class_count)
labels = []
filepaths = []
for row in data:
    labels.append(row['label'])
    filepaths.append(row['image_path'])

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)
# =========================

## load data and  labels
# =========================
data2 = load_path(image_dir2, class_count)
labels = []
filepaths = []
for row in data2:
    labels.append(row['label'])
    filepaths.append(row['image_path'])

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images2 = pd.concat([filepaths, labels], axis=1)
# =========================



In [70]:
## split image
# =========================
# train_df_front, test_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
train_df_front = images.sample(frac=1, random_state=1)
test_df_front = images2
print("Training set label distribution:\n", train_df_front['Label'].value_counts(normalize=False))
print("Test set label distribution:\n", test_df_front['Label'].value_counts(normalize=False))
# =========================

preprocessing_function_chosen_front = tf.keras.applications.resnet50.preprocess_input
preprocessing_function_chosen_side = tf.keras.applications.efficientnet.preprocess_input

# front images
# =========================
train_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=preprocessing_function_chosen_front,
                                                                    validation_split=0.2)
test_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_front)

train_images_front = train_generator_front.flow_from_dataframe(
    dataframe=train_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='training'
)

val_images_front = train_generator_front.flow_from_dataframe(
    dataframe=train_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    seed=42,
    shuffle=False,
    subset='validation'
)

test_images_front = test_generator_front.flow_from_dataframe(
    dataframe=test_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
# =========================


# side images
# =========================
train_df_side = train_df_front.copy()
test_df_side = test_df_front.copy()
train_df_side.loc[:, "Filepath"] = train_df_front["Filepath"].str.replace("front", "side")
test_df_side.loc[:, "Filepath"] = test_df_side["Filepath"].str.replace("front", "side")

train_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=preprocessing_function_chosen_side,
                                                                    validation_split=0.2)
test_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_side)

train_images_side = train_generator_side.flow_from_dataframe(
    dataframe=train_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='training'
)

val_images_side = train_generator_side.flow_from_dataframe(
    dataframe=train_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='validation'
)

test_images_side = test_generator_side.flow_from_dataframe(
    dataframe=test_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
# =========================


Training set label distribution:
 0    126
2     87
1     84
Name: Label, dtype: int64
Test set label distribution:
 2    35
0    34
1    31
Name: Label, dtype: int64
Found 238 validated image filenames belonging to 3 classes.
Found 59 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.
Found 238 validated image filenames belonging to 3 classes.
Found 59 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.


In [71]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Dropout
from tensorflow.keras.applications import ResNet50



def create_front_extract():
    pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

def create_side_extract():
    pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

# input 
input_front = Input(shape=(224, 224, 3), name="AP(Mortise)")
input_side = Input(shape=(224, 224, 3), name="Lateral")

# model
model_front = create_front_extract()
model_side = create_side_extract()

features_front = model_front(input_front)
features_side = model_side(input_side)

features_front_x = tf.keras.layers.Dense(128, activation='relu', name='AP_dense_128')(features_front)
features_front_x = tf.keras.layers.Dense(50, activation='relu', name='AP_dense_50')(features_front_x)

features_side_x = tf.keras.layers.Dense(128, activation='relu', name='Lateral_dense_128')(features_side)
features_side_x = tf.keras.layers.Dense(50, activation='relu', name='Lateral_dense_50')(features_side_x)

fused_features = Concatenate(name="feature_fusion")([features_front_x, features_side_x])
# fused_features = tf.keras.layers.Dense(50, activation='relu', name='fusion_dense_50')(fused_features)
fused_features = Dropout(0.1)(fused_features)

final_output = Dense(class_count, activation='sigmoid', name='output_layer')(fused_features)
multi_view_model = None
multi_view_model = Model(
    inputs=[input_front, input_side],
    outputs=final_output
)
multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

multi_view_model.summary()


Model: "model_821"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AP(Mortise) (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Lateral (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 2048)         23587712    AP(Mortise)[0][0]                
__________________________________________________________________________________________________
efficientnetb0 (Functional)     (None, 1280)         4049571     Lateral[0][0]                    
__________________________________________________________________________________________

In [72]:
def multi_input_generator(front_gen, side_gen):
    while True:
        front_batch, y1 = next(front_gen)
        side_batch, y2 = next(side_gen)
        assert (y1 == y2).all(), "Label mismatch!"  # 確保標籤一致
        yield ([front_batch, side_batch], y1)

train_generator = multi_input_generator(train_images_front, train_images_side)
val_generator = multi_input_generator(val_images_front, val_images_side)

In [73]:
## compile and evaluate
# =========================

print("-------Training " + "_" + concat_type + "-------")
batch_size = 64
## early stop 
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
# multi_view_model.fit(train_generator, validation_data=val_generator, callbacks=[early_stopping], epochs=30,
#                     steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
#                     validation_steps= math.ceil(val_images_front.samples / batch_size))
## no early stop
history = multi_view_model.fit(train_generator, validation_data=val_generator, epochs=30,
                            steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
                            validation_steps= math.ceil(val_images_front.samples / batch_size))


-------Training _concat1_踢盲檢_測盲測-------
Epoch 1/30
4/4 [==============================] - 6s 906ms/step - loss: 1.0916 - accuracy: 0.4454 - val_loss: 0.9702 - val_accuracy: 0.5085
Epoch 2/30
4/4 [==============================] - 2s 540ms/step - loss: 0.9667 - accuracy: 0.5840 - val_loss: 0.8941 - val_accuracy: 0.6441
Epoch 3/30
4/4 [==============================] - 2s 540ms/step - loss: 0.8789 - accuracy: 0.6849 - val_loss: 0.8282 - val_accuracy: 0.6610
Epoch 4/30
4/4 [==============================] - 2s 548ms/step - loss: 0.7848 - accuracy: 0.6975 - val_loss: 0.7623 - val_accuracy: 0.7119
Epoch 5/30
4/4 [==============================] - 2s 539ms/step - loss: 0.7278 - accuracy: 0.7101 - val_loss: 0.7080 - val_accuracy: 0.6949
Epoch 6/30
4/4 [==============================] - 2s 541ms/step - loss: 0.6823 - accuracy: 0.7605 - val_loss: 0.6695 - val_accuracy: 0.6949
Epoch 7/30
4/4 [==============================] - 2s 540ms/step - loss: 0.6310 - accuracy: 0.7899 - val_loss: 0.6414 - v

In [74]:
## save model to this path
# =========================
multi_view_model.save("./weights/"+concat_type+"_" + "_frac.h5")
# =========================


## print results
# =========================
test_generator = multi_input_generator(test_images_front, test_images_side)

batch_size=32
pred = multi_view_model.predict(test_generator,  steps=math.ceil(test_images_front.samples / batch_size))
predicted_labels = np.argmax(pred, axis=1)
# =========================



# create plots for accuracy and save it
# =========================
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_Accuracy.jpeg")
figAcc.savefig(my_file)
plt.clf()
# =========================


## create plots for loss and save it
# =========================
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_"+"_Loss.jpeg")
figAcc.savefig(my_file)
plt.clf()
# =========================


## plot confusion matrix
# =========================
if class_count == 2:
    display_labels = [0, 1]
elif class_count == 3:
    display_labels = [0, 1, 2]
elif class_count == 4:
    display_labels = [0, 1, 2, 3]


cm = confusion_matrix(test_images_front.labels, predicted_labels)
cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = display_labels)
cm_display.plot()
plt.title('Confusion Matrix')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_"+"_Confusion Matrix.jpeg")
figAcc.savefig(my_file)
plt.clf()


d:\anaconda3\envs\bone_20240719\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [75]:
accuracy_score(test_images_front.labels, predicted_labels)

0.8

In [76]:
from keras.preprocessing import image
import matplotlib.cm as cm

def make_heatmap(img_array_list, model, pred_index=None):    
    front_intermediate_model = tf.keras.Model(
        inputs=model.get_layer("resnet50").input,
        outputs=model.get_layer("resnet50").get_layer("conv5_block3_out").output
    )

    side_intermediate_model = tf.keras.Model(
        inputs=model.get_layer("efficientnetb0").input,
        outputs=model.get_layer("efficientnetb0").get_layer("top_conv").output
    )

    fornt_output = side_intermediate_model(img_array_list[0])
    side_output = front_intermediate_model(img_array_list[1])

    last_conv_layer_outputs = [fornt_output, side_output]
    heatmaps=[]
    for conv_output in last_conv_layer_outputs:
        conv_output = conv_output[0]
        heatmap = np.mean(conv_output, axis=-1)

        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)
        heatmaps.append(heatmap)
    return heatmaps

def save_and_display_heatmap(img_path, heatmap, alpha=0.4):
    img = tf.keras.utils.load_img(img_path)
    img = tf.keras.utils.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)

    jet = cm.get_cmap("jet")

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.utils.array_to_img(superimposed_img)
    return superimposed_img

def plot_heatmap_all_branches(image_path, model, save_cam):


    # model = tf.keras.models.load_model(model_path)

    # 載入和處理每個圖像
    image_paths=[image_path, image_path.replace("front", "side")]
    img_array1 = tf.keras.preprocessing.image.img_to_array(image.load_img(image_paths[0], target_size=(224, 224, 3)))
    img_array2 = tf.keras.preprocessing.image.img_to_array(image.load_img(image_paths[1], target_size=(224, 224, 3)))
    # 擴展維度以匹配模型輸入要求
    img_array1 = np.expand_dims(img_array1, axis=0)
    img_array2 = np.expand_dims(img_array2, axis=0)

    img_arrays = [img_array1, img_array2]

    heatmaps = make_heatmap(img_arrays, model)
    # 顯示所有的熱圖
    plt.figure(figsize=(5, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(save_and_display_heatmap(image_paths[0], heatmaps[0]))
    plt.title("AP(Mortise)")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(save_and_display_heatmap(image_paths[1], heatmaps[1]))
    plt.title("Lateral")
    plt.axis("off")

    # 顯示標題
    plt.suptitle(image_path.split('\\')[-1])
    plt.savefig(save_cam)
    plt.close() 

In [77]:
import shutil
def clear_directory(directory_path):
    if os.path.exists(directory_path):
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isdir(file_path):
                shutil.rmtree(file_path)
            else:
                os.remove(file_path) 

combinations = [(0, 0), (1, 1), (2, 2), (1, 2), (2, 1), (0, 1), (1, 0), (2, 0), (0, 2)]
list1 = list(test_df_front['Label'].reset_index(drop=True).astype(int))
# pred
list2 = list(predicted_labels)
for val1, val2 in combinations:
    target_path = save_cam_path + str(val1) + "_" + str(val2)+"//"
    clear_directory(target_path)
    indices = [i for i, (v1, v2) in enumerate(zip(list1, list2)) if v1 == val1 and v2 == val2]
    print(str(val1) + "_" + str(val2)+", count="+str(len(indices))+" :")
    print(indices)
    print()

    # 遍历匹配的索引
    for i in indices:
        tmp = test_df_front.reset_index(drop=True).iloc[i]['Filepath']
        plot_heatmap_all_branches(tmp, multi_view_model, target_path+str(i))



0_0, count=33 :
[35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]

1_1, count=19 :
[73, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 87, 88, 89, 91, 92, 95, 98, 99]

2_2, count=28 :
[0, 1, 3, 5, 6, 7, 9, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34]

1_2, count=9 :
[71, 72, 74, 83, 86, 90, 93, 96, 97]

2_1, count=6 :
[2, 4, 8, 14, 23, 28]

0_1, count=1 :
[39]

1_0, count=3 :
[69, 70, 94]

2_0, count=1 :
[10]

0_2, count=0 :
[]



In [78]:
# chosen_model = "./weights/concat1__frac.h5"


# for i in range(len(wrong1)):
#     # front
#     im_front = test_df_front.reset_index(drop=True).iloc[i]['Filepath']
#     temp_img = image.load_img(im_front, target_size=(224, 224))
#     x = image.img_to_array(temp_img)
#     x = np.expand_dims(x, axis=0)
#     images_front = np.vstack([x])
#     # side
#     im_side = test_df_front.reset_index(drop=True).iloc[i]['Filepath']
#     temp_img = image.load_img(im_side, target_size=(224, 224))
#     x = image.img_to_array(temp_img)
#     x = np.expand_dims(x, axis=0)
#     images_side = np.vstack([x])

#     heatmap = make_heatmap(images_front, images_side, tf.keras.models.load_model(chosen_model))
#     print(f"image path={im}")
#     save_and_display_heatmap(im, heatmap)
#     print("##################################################################################")
# ############################

In [117]:
# 模型答案寫檔
ans_id = [
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001136744F_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002336370E_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001466426H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002181472C_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000113998G_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002628186H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002163835H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000827590J_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001132438C_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002816450D_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001264772C_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002653682D_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001466426H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002075611A_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002663741C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001247832J_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001054203H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001970344H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002294223A_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002660751C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001442921B_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001054203H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000598291C_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000598291C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002538096F_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000733128E_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000047083H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000614556I_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001334670I_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001120714A_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000047083H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002200547G_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001189670B_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001132438C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002622196F_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002549797G_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002149758F_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001189670B_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001243304E_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001292237B_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001309187H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001154623J_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001284091E_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\000899413G_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001780977H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002358396C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001136744F_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\000127859H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000936387F_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\000034129H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002465230A_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001264772C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002615204F_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002124062A_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002451539H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\000833618D_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001292237B_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001960213C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001343885C_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002452272D_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002375803E_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000602012I_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001135192B_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001455017F_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\000673171B_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000746268C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001617424G_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002561319I_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000602012I_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001455017F_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002581753C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002164833J_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000073192A_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002309365E_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001343885C_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001683225A_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000597192G_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002989963G_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001496597A_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000435974E_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000936387F_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002449591I_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002520589E_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000139092H_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001958124J_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000139092H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\001470230I_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002620627J_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001101501D_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\000947946D_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002838799H_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000435974E_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002812483G_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\001120714A_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002606857F_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\002658681C_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000091353B_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\正常\\000091353B_R.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\三踝\\001184477C_L.jpg',
 'E://data_bone//4-a+b_swift_cut_正確//side\\雙踝\\002011721J_R.jpg'
 ]


In [126]:
ans_code = []
for i in range(len(ans_id)):
    # print(ans_id[i])
    matched = False
    for j in range(len(test_df_front)):
        filename = test_df_front['Filepath'].iloc[j].split("\\")[-1]
        if filename in ans_id[i]:
            ans_code.append(predicted_labels[j])
            matched = True
            break


In [129]:
for a in ans_code:
    print(a+1)

1
2
1
3
1
2
2
2
1
3
1
2
1
3
3
2
1
3
2
2
2
1
1
1
2
3
1
3
2
1
1
3
1
1
3
3
3
1
2
1
2
2
3
3
3
3
1
3
1
3
3
1
2
3
3
3
1
3
1
2
2
1
2
1
3
2
3
3
1
1
3
3
1
2
1
3
2
3
1
1
1
3
3
2
2
1
2
3
3
3
3
1
3
1
1
3
1
1
2
2
